In [1]:
from __future__ import division, print_function, absolute_import

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm

# Import MNIST data

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('../mnist', one_hot=True)

from keras import backend as K
#Clear TF memory
cfg = K.tf.ConfigProto()
cfg.gpu_options.allow_growth = True
K.set_session(K.tf.Session(config=cfg))

C:\Users\StrikeWade\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Extracting ../mnist\train-images-idx3-ubyte.gz
Extracting ../mnist\train-labels-idx1-ubyte.gz
Extracting ../mnist\t10k-images-idx3-ubyte.gz
Extracting ../mnist\t10k-labels-idx1-ubyte.gz


Using TensorFlow backend.


In [2]:
#Parameter
learning_rate = 0.001
batch_size = 128
display_step = 20
model_path = '../model/model.ckpt'

#Network Params
n_hidden_1 = 256 #1st layer number of features
n_hidden_2 = 256 #2nd layer number of features
n_input = 784 #mnist image shape 28*28
n_classes = 10

#TF Graph input
x = tf.placeholder(dtype=tf.float32, shape=[None, n_input])
y = tf.placeholder(dtype=tf.float32, shape=[None, n_classes])

#Weights & biases
weights = {
    'h1': tf.Variable(tf.random_normal(shape=[n_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal(shape=[n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal(shape=[n_hidden_2, n_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal(shape=[n_hidden_1])),
    'b2': tf.Variable(tf.random_normal(shape=[n_hidden_2])),
    'out': tf.Variable(tf.random_normal(shape=[n_classes]))
}

In [3]:
#Create model
def multilayer_perceptron(x, weights, biases):
    #Hidden layer 1 with relu activation
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    
    #Hidden layer 2 with relu activation
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    
    #Output layer with linear activation
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    
    return out_layer

In [4]:
#Construct model
y_pred = multilayer_perceptron(x, weights, biases)

#Define loss and optimizer
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

#Init global variables
init = tf.global_variables_initializer()

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



In [5]:
#'Saver' op to save and restore all variables
saver = tf.train.Saver()

In [6]:
#Running 1st Session
print('Starting 1st session:...')
with tf.Session() as sess:
    #Init variable
    sess.run(init)
    
    #Train cycle
    for epoch in range(3):
        avg_loss = 0
        total_batch = int(mnist.train.num_examples / batch_size)
        
        #loop over all batches
        for i in range(total_batch):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            
            #Run optimization op (backprop) and loss op(to get loss value)
            _, l = sess.run([optimizer, loss], feed_dict={x: batch_x, y: batch_y})
            
            #Compute average loss
            avg_loss = l / total_batch
        #Display logs per epoch step
        if epoch % display_step == 0:
            print('Epoch: %d,     average loss= %f' % (epoch+1, avg_loss))
            
    print('\n    First Optimization Completed!!!')
    
    #Test model
    correct_pred = tf.equal(tf.argmax(y_pred, 1), tf.argmax(y, 1))
    
    #Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_pred, dtype=tf.float32))
    
    print('\n    Accuracy on test set: ', accuracy.eval({x: mnist.test.images, y: mnist.test.labels}))
    
    #Save model weights and biases to disk
    save_path = saver.save(sess, model_path)
    print('\n    Model saved in file: %s' % save_path)

Starting 1st session:...
Epoch: 1,     average loss= 0.189256

    First Optimization Completed!!!

    Accuracy on test set:  0.9005

    Model saved in file: ../model/model.ckpt


In [8]:
#Retore and ruung a new session
print('Starting 2nd session:...')
with tf.Session() as sess:
    #Init global variables
    sess.run(init)
    
    #Restore model weights and biases from previously saved model
    load_path = saver.restore(sess, model_path)
    print('\n    Model restored in file: %s' % save_path)
    
    #Resume training
    for epoch in range(10):
        avg_cost = 0
        total_batch = int(mnist.train.num_examples / batch_size) 
        
        #Loop over all batches
        for i in range(total_batch):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            
            #Run optimization op and loss op
            _, l = sess.run([optimizer, loss], feed_dict={x: batch_x, y: batch_y})
            
            #Compute average loss
            avg_loss = l / total_batch
            
        #Display logs per epoch step
        if epoch % display_step == 0:
            print('\n    Epoch: %d,    average loss= %f' % (epoch+1, l))
    print('\n    Second Optimization Completed!!')
    
    #Test model
    correct_pred = tf.equal(tf.argmax(y_pred, 1), tf.argmax(y, 1))
    
    #Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_pred, dtype=tf.float32))
    print('\n    Accuracy on test set: ', accuracy.eval({x: mnist.test.images, y:mnist.test.labels}))

Starting 2nd session:...
INFO:tensorflow:Restoring parameters from ../model/model.ckpt

    Model restored in file: ../model/model.ckpt

    Epoch: 1,    average loss= 32.125679

    Second Optimization Completed!!

    Accuracy on test set:  0.9349
